<a href="https://colab.research.google.com/github/Parinitha-M-Samaga/DL/blob/main/Sentiment_Analysis_using_CNN_and_ML_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


MessageError: Error: credential propagation was unsuccessful

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec

import gensim.downloader as api
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import KeyedVectors

from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import numpy as np

from tensorflow.keras import regularizers



In [ ]:
# Define TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

# Define ModelCheckpoint callback to save the model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="model_checkpoint.h5", save_best_only=True, save_weights_only=False)
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    verbose=1
)

In [ ]:


# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = 0.1  # Percentage of the training data to use for validation
positive_data_file = "rt-polarity.pos"
negative_data_file = "rt-polarity.neg"

# Model Hyperparameters
embedding_dim = 100
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.55
l2_reg_lambda = 0.0

# Training parameters
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

# Misc Parameters
allow_soft_placement = True
log_device_placement = False

# Data Preparation
# ==================================================

# Load data
print("Loading data...")
positive_examples = list(open(positive_data_file, "r").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_data_file, "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
y = [1 if i < len(positive_examples) else 0 for i in range(len(x_text))]
X_train, X_test, y_train, y_test = train_test_split(x_text,y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(len(X_train),len(X_test),len(y_train),len(y_test))


sentences = [word_tokenize(sentence) for sentence in X_train]
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
print("adhi",len(X_train))
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
print("adhi1",len(X_train))
# Train the Word2Vec model


w2v_model = Word2Vec(sentences,vector_size=100, window=5, min_count=5, workers=4)
#train=[[0 0 0 0  2 3 4 ]]

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

def create_text_cnn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=True))
    model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, padding='valid', activation='relu'))
    model.add(tf.keras.layers.GlobalMaxPooling1D())
    model.add(tf.keras.layers.Dropout(rate=dropout_keep_prob))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2, activation='softmax'))  # Softmax layer for classification
    return model

textcnn_model = create_text_cnn()
textcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Train the TextCNN model
history = textcnn_model.fit(X_train, y_train,
                             validation_data=(X_val, y_val),
                             batch_size=batch_size,
                             epochs=num_epochs, callbacks=[early_stopping_callback]
                            )

# Remove the last layers of the TextCNN model
for layer in textcnn_model.layers[:-3]:
    layer.trainable = False
from tensorflow.keras.models import Model

# Define a new model that outputs the Flatten layer's activations
feature_extraction_model = Model(inputs=textcnn_model.input, outputs=textcnn_model.layers[-2].output)

# Extract features from the Flatten layer
X_train_features = feature_extraction_model.predict(X_train)
X_test_features = feature_extraction_model.predict(X_test)

print(X_train_features)
# Train logistic regression model on the extracted features
logreg_model = LogisticRegression(penalty='l2', C=0.8)
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Loading data...
8635 1067 8635 1067
adhi 8635
adhi1 8635
Epoch 1/200
135/135 [==============================] - 11s 78ms/step - loss: 0.7033 - accuracy: 0.5266 - val_loss: 0.6777 - val_accuracy: 0.6104
Epoch 2/200
135/135 [==============================] - 4s 27ms/step - loss: 0.6504 - accuracy: 0.6293 - val_loss: 0.6141 - val_accuracy: 0.6938
Epoch 3/200
135/135 [==============================] - 3s 24ms/step - loss: 0.3906 - accuracy: 0.8471 - val_loss: 0.5915 - val_accuracy: 0.7146
Epoch 4/200
135/135 [==============================] - 2s 12ms/step - loss: 0.1382 - accuracy: 0.9565 - val_loss: 0.7315 - val_accuracy: 0.7156
Epoch 5/200
135/135 [==============================] - 1s 10ms/step - loss: 0.0658 - accuracy: 0.9812 - val_loss: 0.8556 - val_accuracy: 0.7115
Epoch 6/200
135/135 [==============================] - 1s 8ms/step - loss: 0.0334 - accuracy: 0.9914 - val_loss: 1.0024 - val_accuracy: 0.7135
Epoch 7/200
135/135 [==============================] - 1s 7ms/step - loss: 0.02

In [ ]:
# Train logistic regression model on the extracted features
logreg_model = LogisticRegression()
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

accuracy2 = logreg_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy2:.4f}")

Test Accuracy: 0.7160
Train Accuracy: 1.0000


In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=85, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)
random_forest_model.fit(X_train_features, y_train)

# Evaluate Random Forest model
accuracy = random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
accuracy3 = random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy3:.4f}")

Test Accuracy: 0.7029
Train Accuracy: 1.0000


In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7095
Train Accuracy: 0.9999


In [ ]:


# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = 0.1  # Percentage of the training data to use for validation
positive_data_file = "rt-polarity.pos"
negative_data_file = "rt-polarity.neg"

# Model Hyperparameters
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size = 64
num_epochs = 19
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

# Misc Parameters
allow_soft_placement = True
log_device_placement = False

# Data Preparation
# ==================================================

# Load data
print("Loading data...")
positive_examples = list(open(positive_data_file, "r").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_data_file, "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
y = [1 if i < len(positive_examples) else 0 for i in range(len(x_text))]
X_train, X_test, y_train, y_test = train_test_split(x_text,y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(len(X_train),len(X_test),len(y_train),len(y_test))


sentences = [word_tokenize(sentence) for sentence in X_train]
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
print("adhi",len(X_train))
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
print("adhi1",len(X_train))
# Train the Word2Vec model


w2v_model = Word2Vec(sentences,vector_size=100, window=5, min_count=5, workers=4)
#train=[[0 0 0 0  2 3 4 ]]

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

def create_text_cnn():
    # Define inputs for each channel
    input_static = tf.keras.layers.Input(shape=(max_length,))
    input_non_static = tf.keras.layers.Input(shape=(max_length,))

    # Static channel: Use pre-trained embedding matrix and set trainable=False
    embedding_static = tf.keras.layers.Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], input_length=max_length, trainable=False)(input_static)

    # Non-static channel: Use separate embedding layer with trainable=True
    embedding_non_static = tf.keras.layers.Embedding(vocab_size, embedding_matrix.shape[1], input_length=max_length, trainable=True)(input_non_static)

    # Convolution and max-pooling layers for both channels with specified filter windows and 100 feature maps
    conv_static_3 = tf.keras.layers.Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu')(embedding_static)
    conv_static_4 = tf.keras.layers.Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu')(embedding_static)
    conv_static_5 = tf.keras.layers.Conv1D(filters=100, kernel_size=5, padding='valid', activation='relu')(embedding_static)

    conv_non_static_3 = tf.keras.layers.Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu')(embedding_non_static)
    conv_non_static_4 = tf.keras.layers.Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu')(embedding_non_static)
    conv_non_static_5 = tf.keras.layers.Conv1D(filters=100, kernel_size=5, padding='valid', activation='relu')(embedding_non_static)

    maxpool_static_3 = tf.keras.layers.GlobalMaxPooling1D()(conv_static_3)
    maxpool_static_4 = tf.keras.layers.GlobalMaxPooling1D()(conv_static_4)
    maxpool_static_5 = tf.keras.layers.GlobalMaxPooling1D()(conv_static_5)

    maxpool_non_static_3 = tf.keras.layers.GlobalMaxPooling1D()(conv_non_static_3)
    maxpool_non_static_4 = tf.keras.layers.GlobalMaxPooling1D()(conv_non_static_4)
    maxpool_non_static_5 = tf.keras.layers.GlobalMaxPooling1D()(conv_non_static_5)

    # Concatenate the outputs of both channels
    merged_static = tf.keras.layers.Concatenate()([maxpool_static_3, maxpool_static_4, maxpool_static_5])
    merged_non_static = tf.keras.layers.Concatenate()([maxpool_non_static_3, maxpool_non_static_4, maxpool_non_static_5])

    # Concatenate the outputs of both channels
    merged = tf.keras.layers.Concatenate()([merged_static, merged_non_static])

    # Dropout layer for regularization with dropout rate of 0.5
    dropout = tf.keras.layers.Dropout(0.5)(merged)

    # Output layer
    output = tf.keras.layers.Dense(2, activation='softmax')(dropout)

    # Define the model with two inputs and one output
    model = tf.keras.Model(inputs=[input_static, input_non_static], outputs=output)

    return model


textcnn_model = create_text_cnn()
textcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Assuming X_train_static and X_train_non_static are the input data for static and non-static channels, respectively
history = textcnn_model.fit([X_train, X_train], y_train,
                            validation_data=([X_val, X_val], y_val),
                            batch_size=batch_size,
                            epochs=num_epochs,
                            callbacks=[early_stopping_callback])


# Remove the last layers of the TextCNN model
for layer in textcnn_model.layers[:-3]:
    layer.trainable = False
from tensorflow.keras.models import Model

# Define a new model that outputs the Flatten layer's activations
feature_extraction_model = Model(inputs=textcnn_model.input, outputs=textcnn_model.layers[-2].output)

# Extract features from the Flatten layer

X_train_features= feature_extraction_model.predict([X_train, X_train])

X_test_features = feature_extraction_model.predict([X_test,X_test])

print(X_train_features)



Loading data...
8635 1067 8635 1067
adhi 8635
adhi1 8635
Epoch 1/19
135/135 [==============================] - 19s 118ms/step - loss: 0.6978 - accuracy: 0.5458 - val_loss: 0.6606 - val_accuracy: 0.5792
Epoch 2/19
135/135 [==============================] - 8s 56ms/step - loss: 0.4930 - accuracy: 0.7651 - val_loss: 0.4857 - val_accuracy: 0.7719
Epoch 3/19
135/135 [==============================] - 4s 30ms/step - loss: 0.2424 - accuracy: 0.9062 - val_loss: 0.5531 - val_accuracy: 0.7729
Epoch 4/19
135/135 [==============================] - 4s 32ms/step - loss: 0.1003 - accuracy: 0.9724 - val_loss: 0.7023 - val_accuracy: 0.7563
Epoch 5/19
135/135 [==============================] - 2s 13ms/step - loss: 0.0393 - accuracy: 0.9899 - val_loss: 0.8263 - val_accuracy: 0.7479
Epoch 6/19
135/135 [==============================] - 2s 15ms/step - loss: 0.0155 - accuracy: 0.9976 - val_loss: 0.9580 - val_accuracy: 0.7406
Epoch 7/19
135/135 [==============================] - 2s 13ms/step - loss: 0.0088 -

In [ ]:
len(X_train)

8635

In [ ]:
# Train logistic regression model on the extracted features with L2 regularization
logreg_model = LogisticRegression(penalty='l2', C=0.5)
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

accuracy2 = logreg_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy2:.4f}")

Test Accuracy: 0.7357
Train Accuracy: 1.0000


In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=90,max_depth=5, min_samples_split=4, min_samples_leaf=2, max_features='sqrt', random_state=42)
random_forest_model.fit(X_train_features, y_train)

# Evaluate Random Forest model
accuracy = random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
accuracy3 = random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy3:.4f}")

Test Accuracy: 0.7310
Train Accuracy: 1.0000


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Instantiate the random forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_features, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Instantiate the random forest model with the best parameters
best_random_forest_model = RandomForestClassifier(**best_params, random_state=42)

# Fit the model to the training data
best_random_forest_model.fit(X_train_features, y_train)

# Evaluate the model on the test data
accuracy = best_random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Evaluate the model on the training data
accuracy_train = best_random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy_train:.4f}")


Fitting 3 folds for each of 243 candidates, totalling 729 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='linear', C=0.7,gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7376
Train Accuracy: 1.0000


In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='rbf',gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7357
Train Accuracy: 1.0000


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Instantiate the SVM classifier
svm_model = SVC(random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_features, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Instantiate the SVM model with the best parameters
best_svm_model = SVC(**best_params, random_state=42)

# Fit the model to the training data
best_svm_model.fit(X_train_features, y_train)

# Evaluate the model on the test data
accuracy_test = best_svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy_test:.4f}")

# Evaluate the model on the training data
accuracy_train = best_svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy_train:.4f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize kNN classifier
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train kNN model
knn_model.fit(X_train_features, y_train)

# Evaluate kNN model
accuracy_test = knn_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy_test:.4f}")

# Evaluate kNN model on the training data
accuracy_train = knn_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy_train:.4f}")
